2.7 Extension de tipos de datos

Pandas se construyó originalmente sobre las capacidades presentes en NumPy, una librería de cálculo de arrays utilizada principalmente para trabajar con datos numéricos. Muchos conceptos de pandas, como los datos que faltan, se implementaron utilizando lo que estaba disponible en NumPy mientras se intentaba maximizar la compatibilidad entre las bibliotecas que utilizaban conjuntamente NumPy y Pandas.

Basarse en NumPy conllevó una serie de deficiencias, como:

El manejo de datos faltantes para algunos tipos de datos numéricos, como enteros y booleanos, era incompleto. Como resultado, cuando se introducían datos perdidos en tales datos, pandas convertía el tipo de datos a float64 y utilizaba np.nan para representar valores nulos. Esto tenía efectos agravantes al introducir problemas sutiles en muchos algoritmos de pandas.

Los conjuntos de datos con muchos datos de cadenas eran costosos computacionalmente y utilizaban mucha memoria.

Algunos tipos de datos, como intervalos de tiempo, timedeltas, y timestamps con zonas horarias, no podían ser soportados eficientemente sin usar arrays de objetos Python computacionalmente caros.

Más recientemente, pandas ha desarrollado un sistema de tipos de extensión que permite añadir nuevos tipos de datos aunque no estén soportados nativamente por NumPy. Estos nuevos tipos de datos pueden ser tratados como de primera clase junto con los datos procedentes de arrays NumPy.

Veamos un ejemplo en el que creamos una Serie de enteros con un valor perdido:

In [1]:
import pandas as pd
import numpy as np

In [7]:
s = pd.Series([1, None, 3, 4, None])
s

0    1.0
1    NaN
2    3.0
3    4.0
4    NaN
dtype: float64

In [8]:
s_2 = pd.Series([1,3,4])
s_2

0    1
1    3
2    4
dtype: int64

Por defecto, pandas tratará de inferir el tipo de datos de la serie. Dado que hay un None en la lista, pandas convertirá automáticamente el tipo de datos a float64 porque el tipo float en pandas puede manejar valores nulos (NaN), mientras que el tipo int tradicional no puede.

In [9]:
s.dtype

dtype('float64')

Principalmente por razones de retrocompatibilidad, Series utiliza el comportamiento heredado de usar un tipo de datos float64 y np.nan para el valor perdido. Podríamos crear esta Serie en su lugar usando pandas.Int64Dtype:

In [12]:
s_1 = pd.Series([1, None, 3, 4, None],dtype=pd.Int64Dtype())
s_1

0       1
1    <NA>
2       3
3       4
4    <NA>
dtype: Int64

Aquí, especificamos explícitamente que queremos usar el tipo de datos Int64 de pandas, que es un tipo de datos de enteros que soporta valores nulos. Esto permite que la serie mantenga su tipo de datos como enteros (Int64), incluso con la presencia de valores nulos. Los valores nulos se representan con , que es un valor nulo especial compatible con el tipo Int64.

In [13]:

s_1.dtype

Int64Dtype()

La salida <NA> indica que falta un valor para un array de tipo extensión. Esto utiliza el valor centinela especial pandas.NA:

In [16]:
s_1[1]

<NA>

In [17]:
s_1[1] is pd.NA

True

In [18]:
s[2] is pd.NA

False

También podríamos haber utilizado la abreviatura "Int64" en lugar de pd.Int64Dtype() para especificar el tipo. La mayúscula es necesaria, de lo contrario será un tipo sin extensión basado en NumPy:

In [20]:
s_3 = pd.Series([1,None, 3, 4, None], dtype="Int64")
s_3                

0       1
1    <NA>
2       3
3       4
4    <NA>
dtype: Int64

Pandas también tiene un tipo de extensión especializado para datos de cadena que no utiliza arrays de objetos NumPy (requiere la biblioteca pyarrow, que puede que necesite instalar por separado):

In [21]:
se = pd.Series(['one', None, 'three','four',None])
se

0      one
1     None
2    three
3     four
4     None
dtype: object

In [22]:
s_4 = pd.Series(['one', None, 'three','four',None], dtype=pd.StringDtype())
s_4

0      one
1     <NA>
2    three
3     four
4     <NA>
dtype: string

Estos arrays de cadenas suelen utilizar mucha menos memoria y, con frecuencia, son más eficientes desde el punto de vista computacional para realizar operaciones en grandes conjuntos de datos.

En la siguiente Tabla figura una lista de algunos de los tipos de extensión disponibles.

BooleanDtype: Datos booleanos anulables, utilice "boolean" al pasarlos como cadena.

CategoricalDtype: Tipo de dato categórico, utilice "category" cuando pase como cadena

DatetimeTZDtype: Datetime with time zone

Float32Dtype: Coma flotante anulable 32-bit , use "Float32" cuando pase como cadena.

Float64Dtype: Coma flotante anulable de 64 bits, utilice "Float64" al pasar como cadena.

Int8Dtype: Entero con signo anulable de 8 bits, utilice "Int8" al pasarlo como cadena.

Int16Dtype: Entero con signo anulable de 16 bits, utilice "Int16" al pasarlo como cadena.

Int32Dtype: Entero con signo anulable de 32 bits, utilice "Int32" al pasarlo como cadena.

Int64Dtype: Entero con signo anulable de 64 bits, utilice "Int64" al pasarlo como cadena.

UInt8Dtype: Entero sin signo anulable de 8 bits, utilice "UInt8" al pasarlo como cadena.

UInt16Dtype: Entero sin signo anulable de 16 bits, utilice "UInt16" al pasarlo como cadena.

UInt32Dtype: Entero sin signo de 32 bits anulable, utilice "UInt32" al pasarlo como cadena.

UInt64Dtype: Entero sin signo de 64 bits anulable, use "UInt64" cuando pase como cadena.



Los tipos de extensión pueden pasarse al método Series astype, lo que permite convertirlos fácilmente como parte del proceso de limpieza de datos:

In [24]:
df = pd.DataFrame({"A":[1,None, 3, 4, None],
                   "B":['one',None,'three','four',None],
                   "C":[True, False, None,True, None]})
df

,A,B,C
0,1.0,one,True
1,NaN,None,False
2,3.0,three,None
3,4.0,four,True
4,NaN,None,None


In [27]:
df["A"] = df["A"].astype("Int64")

In [28]:
df["B"] = df["B"].astype("string")

In [29]:
df["C"] = df["C"].astype("boolean")

In [30]:
df

,A,B,C
0,1,one,True
1,<NA>,<NA>,False
2,3,three,<NA>
3,4,four,True
4,<NA>,<NA>,<NA>


2.8 Manipulación de cadenas (string)

Python ha sido durante mucho tiempo un lenguaje popular de manipulación de datos en bruto, en parte debido a su facilidad de uso para el procesamiento de cadenas y texto. La mayoría de las operaciones de texto se simplifican con los métodos incorporados en el objeto string. Para la comparación de patrones y manipulaciones de texto más complejas, pueden ser necesarias expresiones regulares. Pandas se suma a la mezcla al permitirle aplicar expresiones de cadena y regulares de forma concisa en arrays enteros de datos, manejando además la molestia de los datos que faltan.

Métodos de objetos de cadena incorporados en Python

En muchas aplicaciones de manipulación de cadenas y scripts, los métodos de cadena incorporados son suficientes. Por ejemplo, una cadena separada por comas puede dividirse en trozos con split:

In [31]:
val = "a,n, gelica"
val.split(",")

['a', 'n', ' gelica']

split se combina a menudo con strip para recortar los espacios en blanco (incluidos los saltos de línea):

In [32]:
pieces = [x.strip() for x in val.split(",")]
pieces

['a', 'n', 'gelica']

Estas subcadenas (substrings) podrían concatenarse con un delimitador de dos puntos utilizando la suma:

In [33]:
first, second, third = pieces
first + "::" + second +"::" + third

'a::n::gelica'

Pero éste no es un método genérico práctico. Una forma más rápida y pitónica es pasar una lista o tupla al método join sobre la cadena "::":

In [34]:
"::".join(pieces)

'a::n::gelica'

Otros métodos se ocupan de localizar subcadenas. Utilizar la palabra clave in de Python es la mejor forma de detectar una subcadena, aunque también se pueden utilizar index y find:

In [35]:
"gelica" in val

True

In [36]:
val.index(",")

1

In [38]:
val.find(":")

-1

Tenga en cuenta que la diferencia entre find e index es que index lanza una excepción si no se encuentra la cadena (en lugar de devolver -1):

In [ ]:
val.index(":")

Por su parte, count devuelve el número de apariciones de una determinada subcadena:

In [40]:
val.count(",")

2

replace sustituirá las ocurrencias de un patrón por otro. También se suele utilizar para eliminar patrones, pasando una cadena vacía:

In [41]:
val.replace(",","::")

'a::n:: gelica'

In [42]:
val.replace(",","")

'an gelica'